### import

In [1]:
## 
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import json

##
from DLinear import LTSF_DLinear
from DLinear import moving_avg
from utils import standardization
from utils import time_slide_df
from utils import Data
from utils import data_split
from utils import date_range_to_numeric

/Users/hyeongyukang/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[20231101, 20231102, 20231103, 20231104, 20231105]


### load dataset and split by item, corporation, location

In [2]:
# data_loading
data = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")


data_list = data_split(data)
test_list = data_split(test)

unique_code = ['BC_C_J', 'TG_B_J', 'CR_B_J', 'RD_E_S', 'BC_A_J', 'CB_F_J', 'RD_D_J', 'TG_A_S', 'BC_E_S', 'CR_D_J', 'BC_A_S', 'BC_B_S', 'TG_E_J', 
               'CR_E_S', 'RD_F_J', 'BC_E_J', 'TG_A_J', 'CR_C_J', 'CR_D_S', 'TG_C_J', 'CB_A_S', 'TG_D_J', 'CR_E_J', 'RD_C_S', 'BC_C_S', 'CB_E_J', 
               'RD_E_J', 'BC_D_J', 'CR_A_J', 'TG_E_S', 'TG_C_S', 'TG_D_S', 'RD_A_S', 'RD_A_J', 'RD_D_S', 'TG_B_S', 'CB_D_J', 'CB_A_J', 'BC_B_J']

## CR_D_S, CB_A_S, BC_B_S, BC_C_S, CR_E_S, RD_C_S : 3월에 일반적으로 0을 가짐

In [3]:
## prepare dataset for training
dataset_code = unique_code[1]
train_df = data_list[f'data_{dataset_code}'].reset_index(drop=True)
test_df = test_list[f'data_{dataset_code}'].reset_index(drop=True)
print(len(train_df))

# preprocess timestamp
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'])
test_df['timestamp'] = pd.to_datetime(test_df['timestamp'])

1523


In [11]:
## paramaters
window_size = 7
forcast_size= 1
batch_size = 32
targets = 'price(원/kg)'
date = 'timestamp'
not_col = 'timestamp'

In [13]:
def time_slide_df1(df, window_size, forcast_size, date, target):
    df_ = df.copy()
    data_list = []
    dap_list = []
    date_list = []
    for idx in range(0, df_.shape[0]-window_size-forcast_size+1): # range(0,1516)
        x = df_.loc[idx:idx+window_size-1, target].values.reshape(window_size, 1)
        y = df_.loc[idx+window_size:idx+window_size+forcast_size-1, target].values
        date_ = df_.loc[idx+window_size:idx+window_size+forcast_size-1, date].values
        data_list.append(x)
        dap_list.append(y)
        date_list.append(date_)
    return np.array(data_list, dtype='float32'), np.array(dap_list, dtype='float32'), np.array(date_list)

In [14]:
train_x, train_y, train_date = time_slide_df1(train_df, window_size, forcast_size, date, targets)
# train_x.shape : (1486, 31, 1)

In [27]:
train_y[-7:].reshape(1,-1)
np.append(train_y[-7:].reshape(1,-1),1)

array([2.412e+03, 0.000e+00, 3.540e+03, 3.141e+03, 6.382e+03, 3.558e+03,
       3.470e+03, 1.000e+00])

In [ ]:
input = torch.Size([32, 7, 1])
ouput = torch.Size([1, 1, 1])

valid_y[-7:]
all = [1,2,3,4,5,6,7]
pred = []
torch.Tensor(all)
model(all)-> output1(3월 4일 결과 출력)

all.append(output1.item())
pred.append(output1.item())

all = [1,2,3,4,5,6,7,output1]
all.pop(0)


In [ ]:
data = train_y[-7:].reshape(1,-1)
pred = []
for i in range(28):
    pred_input = torch.Tensor(data).reshape(1,7,1)
    pred_output = model(pred_input)

    np.append(data,pred_output.item())
    pred.append(pred_output.item())
    data.pop(0)

In [ ]:
def apply_function_and_slide(data, window_size):
    input = []  # 결과 값을 저장할 리스트

    for i in range(len(data)):
        window = data[i:i+window_size]  # 현재 윈도우
        if len(window) == window_size:
            # 윈도우 크기가 7인 경우 함수를 적용하고 결과를 결과 리스트에 추가
            output = model(window)
            input.append(output)

    return input

In [ ]:


pred = {}

# for i in tqdm(range(len(unique_code))):
for i in tqdm(range(len(unique_code))):

    ## prepare dataset for training
    dataset_code = unique_code[i]

    train_df = data_list[f'data_{dataset_code}'].reset_index(drop=True)
    test_df = test_list[f'data_{dataset_code}'].reset_index(drop=True)
    print(len(train_df))

    # train_df = train_df.drop(['ID', 'item', 'corporation', 'location'], axis=1)
    # test_df = test_df.drop(['ID', 'item', 'corporation', 'location'], axis=1)



    train_df['timestamp'] = pd.to_datetime(train_df['timestamp'])
    test_df['timestamp'] = pd.to_datetime(test_df['timestamp'])
    print("Dataset prepared")
    print()

    ## paramaters
    window_size = 31
    forcast_size= 7
    batch_size = 32
    targets = 'price(원/kg)'
    date = 'timestamp'
    not_col = 'timestamp'

    print("Start preprocessing")
    print()
    # train_df_fe, test_df_fe, mean_, std_ = standardization(train_df, test_df, not_col, targets)
    # train_x, train_y, train_date = time_slide_df(train_df_fe, window_size, forcast_size, date, targets)
    # test_x, test_y, test_date = time_slide_df(test_df_fe, window_size, forcast_size, date, targets)

    train_x, train_y, train_date = time_slide_df(train_df, window_size, forcast_size, date, targets)
    # test_x, test_y, test_date = time_slide_df(test_df, window_size, forcast_size, date, targets)


    train_ds = Data(train_x[:1400], train_y[:1400])
    valid_ds = Data(train_x[1400:], train_y[1400:])
    # test_ds = Data(test_x, test_y)

    train_dl = DataLoader(train_ds, batch_size = batch_size, shuffle=True,)
    valid_dl = DataLoader(valid_ds, batch_size = train_x[1200:].shape[0], shuffle=False)
    # test_dl  = DataLoader(test_ds,  batch_size = test_x.shape[0], shuffle=False)
    print("Success preprocessing")


    print("Paramater for training")
    print()
    train_loss_list = []
    valid_loss_list = []
    test_loss_list = []
    epochs = 100
    lr = 0.001
    DLinear_model = LTSF_DLinear(
                                window_size=window_size,
                                forcast_size=forcast_size,
                                kernel_size=25,
                                individual=False,
                                feature_size=1,
                                )
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(DLinear_model.parameters(), lr=lr)
    max_loss = 999999999

    print("strat training")
    print()


    for epoch in tqdm(range(1, epochs+1)):
        loss_list = []
        DLinear_model.train()
        for batch_eeeeidx, (data, target) in enumerate(train_dl):
            optimizer.zero_grad()
            output = DLinear_model(data)
            loss = criterion(output, target.unsqueeze(-1))
            loss.backward()
            optimizer.step()
            loss_list.append(np.sqrt(loss.item()))  
        train_loss_list.append(np.sqrt(np.mean(loss_list)))

        DLinear_model.eval()
        with torch.no_grad():
            for data, target in valid_dl:
                output = DLinear_model(data)
                # if epoch == 100:
                #     temp = output.reshape(1,-1)
                #     temp1 = temp[:,-61:].reshape(1,61,1)
                #     pred[dataset_code] = DLinear_model(temp1).reshape(1,-1).numpy()
                valid_loss = np.sqrt(criterion(output, target.unsqueeze(-1)).item())
                valid_loss_list.append(np.sqrt(valid_loss.item()))
            
            # for data, target in test_dl:
            #     output = DLinear_model(data)
            #     test_loss = criterion(output, target.unsqueeze(-1))
            #     test_loss_list.append(test_loss)

        # if valid_loss < max_loss:
        #     torch.save(DLinear_model, f'DLinear_model_{dataset_code}.pth')
        #     max_loss = valid_loss
        #     # print("valid_loss={:.3f}, test_los{:.3f}, Model Save".format(valid_loss, test_loss))
        #     print("valid_loss={:.3f}, Model Save".format(valid_loss))
        #     dlinear_best_epoch = epoch
        #     dlinear_best_train_loss = np.mean(loss_list)
        #     dlinear_best_valid_loss = np.mean(valid_loss.item())

        # print("epoch = {}, train_loss : {:.3f}, valid_loss : {:.3f}, test_loss : {:.3f}".format(epoch, np.mean(loss_list), valid_loss, test_loss))
        print("epoch = {}, train_loss : {:.3f}, valid_loss : {:.3f}".format(epoch, np.mean(loss_list), valid_loss))




# ## Storing result

# date_col = []
# date = 20230304
# for i in range(28):
#     date_col.append(date + i)
# print(date_col)



# result = []
# for i in range(len(unique_code)):
#     pred_adj = pred[unique_code[i]][0][3:]
#     for j in range(28):
#         code = unique_code[i]
#         final = pred_adj[j]
#         if date_col[j] in [20230305, 20230312, 20230319, 20230326]:
#             final = 0
#         else:
#             final = final

#         code_time = f"{unique_code[i]}_{date_col[j]}"
#         result.append([code_time, final])
    

# result_df = pd.DataFrame({'ID': result[:, 0], 'pred': result[:, 1]})
# submission = pd.read_csv("data/sample_submission.csv")


# # 'submission' 데이터프레임과 'result_df' 데이터프레임을 'ID'를 기준으로 병합
# final_submission = submission.merge(result_df, on='ID', how='left')

# # 'pred' 값을 'answer' 열에 복사
# final_submission['answer'] = final_submission['pred']

# # 'pred' 열 삭제
# final_submission.drop('pred', axis=1, inplace=True)

# # 결과 데이터프레임 출력
# print(final_submission.head())


# # 업데이트된 데이터프레임을 새로운 CSV 파일로 저장
# final_submission.to_csv('submission_vanila.csv', index=False)
